<a href="https://colab.research.google.com/github/JavierCuellar5/catsVSdogs/blob/main/Del_modelo_a_la_aplicaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Del modelo a la aplicación

###En este sprint vamos a dar un paso más allá y vamos a construir una aplicación real de Machine Learning en Python para resolver un problema de clasificación de imágenes "Perro/Gato".

##Configuración inicial: Activamos el modo legacy de Keras (Keras 2).

In [24]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

###Carga del modelo y etiquetas: Cargamos un modelo pre-entrenado llamado keras_model.h5 y las etiquetas de las clases (gatos y perros) desde Google Drive.

In [25]:
from tensorflow import keras

#Carga el modelo/content/drive/MyDrive/keras_model.h5
model = keras.models.load_model("/content/drive/MyDrive/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/labels.txt", "r").readlines()

print(nombre_clases)


['0 gatos\n', '1 perros\n']


##Predicción de una sola imagen: Probamos el modelo con una imagen de gato (cat.990.jpg)

In [26]:
from PIL import Image, ImageOps
image= Image.open("/content/drive/MyDrive/cat.990.jpg").convert("RGB")


###Cargamos y preprocesamos la imagen (cambio de tamaño y normalización)

In [27]:
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)


In [28]:
import numpy as np
imagen_array = np.asarray(image)


In [29]:
normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1


###El modelo predijo correctamente que era un gato con una confianza del 95.36%.

##Preparación para la predicción en lote: Intentamos procesar varias imágenes de prueba de carpeta /content/drive/MyDrive/test.

In [30]:
lista_archivos = os.listdir("/content/drive/MyDrive/test")

###Inicializamos variables para contar las predicciones totales, los aciertos y las predicciones incorrectas.

In [31]:
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

###Iniciamos un bucle para recorrer cada imagen en la carpeta de prueba, determinar la etiqueta esperada (gato o perro según el nombre del archivo), e intentar predecir su clase y probabilidad.

In [32]:
from PIL import Image, ImageOps
import numpy as np

def predecir_imagen(ruta_imagen):
    # Cargar y preprocesar la imagen
    image = Image.open(ruta_imagen).convert("RGB")
    size = (224, 224) # Usamos el tamaño definido previamente
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    imagen_array = np.asarray(image)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalizada_imagen_array

    # Realizar la predicción
    prediction = model.predict(data)
    index = np.argmax(prediction)
    predicted_class_name_raw = nombre_clases[index]
    confidence_score = prediction[0][index]

    # Limpiar el nombre de la clase (quitando el número y el salto de línea)
    etiqueta_predicha = predicted_class_name_raw[2:].strip()

    return etiqueta_predicha, confidence_score

for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "gatos"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "perros"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)

    print(f"Archivo: {nombre_archivo}, Predicción: {etiqueta_predicha}, Probabilidad: {probabilidad}, Clase: {etiqueta_esperada}")

1/1 [==============================] - 1s 791ms/step
Archivo: dog.9799.jpg, Predicción: perros, Probabilidad: 0.9995120763778687, Clase: perros
1/1 [==============================] - 0s 21ms/step
Archivo: dog.9781.jpg, Predicción: perros, Probabilidad: 0.9924874305725098, Clase: perros
1/1 [==============================] - 0s 22ms/step
Archivo: dog.9808.jpg, Predicción: perros, Probabilidad: 0.6999475955963135, Clase: perros
1/1 [==============================] - 0s 24ms/step
Archivo: dog.9785.jpg, Predicción: perros, Probabilidad: 0.9897101521492004, Clase: perros
1/1 [==============================] - 0s 49ms/step
Archivo: dog.9777.jpg, Predicción: perros, Probabilidad: 0.9970042109489441, Clase: perros
1/1 [==============================] - 0s 20ms/step
Archivo: dog.9797.jpg, Predicción: perros, Probabilidad: 0.9968665242195129, Clase: perros
1/1 [==============================] - 0s 21ms/step
Archivo: dog.9780.jpg, Predicción: perros, Probabilidad: 0.9985612034797668, Clase: perro

###Finalmente, vamos a calcular las métricas de rendimiento del modelo y generar un informe con los resultados obtenidos. Para ello, usaremos los contadores inicializados en la sección anterior y generaremos un informe

In [33]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1

###Generamos el informe con los resultados obtenidos.

In [34]:
informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 400
Aciertos: 389
Precisión: 97.25%
Probabilidad media de aciertos: 95.55%

Predicciones incorrectas:
Archivo: dog.9854.jpg, Predicción: gatos, Probabilidad: 64.12%
Archivo: dog.9868.jpg, Predicción: gatos, Probabilidad: 57.34%
Archivo: dog.9911.jpg, Predicción: gatos, Probabilidad: 83.74%
Archivo: dog.9935.jpg, Predicción: gatos, Probabilidad: 57.00%
Archivo: cat.9882.jpg, Predicción: perros, Probabilidad: 72.70%
Archivo: cat.9897.jpg, Predicción: perros, Probabilidad: 75.26%
Archivo: cat.9947.jpg, Predicción: perros, Probabilidad: 76.91%
Archivo: cat.9984.jpg, Predicción: perros, Probabilidad: 61.17%
Archivo: cat.9974.jpg, Predicción: perros, Probabilidad: 66.14%
Archivo: cat.9966.jpg, Predicción: perros, Probabilidad: 52.49%
Archivo: cat.9960.jpg, Predicción: perros, Probabilidad: 81.32%

